# Radial FLASH

Qualitative imaging with a golden radial FLASH (spoiled gradient echo) sequence.

### Imports

In [ ]:
import tempfile
from pathlib import Path

import matplotlib.pyplot as plt
import MRzeroCore as mr0
import numpy as np
from einops import rearrange
from mrpro.algorithms.reconstruction import DirectReconstruction
from mrpro.data import KData
from mrpro.data import SpatialDimension
from mrpro.data.traj_calculators import KTrajectoryRadial2D
from mrpro.operators.models import SpoiledGRE

from mrseq.scripts.radial_flash import main as create_seq
from mrseq.utils import sys_defaults

### Settings
We are going to use a numerical phantom with a matrix size of 128 x 128.

In [ ]:
image_matrix_size = [128, 128]

tmp = tempfile.TemporaryDirectory()
fname_seq = Path(tmp.name) / 'radial_flash.seq'
fname_mrd = Path(tmp.name) / 'radial_flash.mrd'

### Create the digital phantom
We use the standard Brainweb phantom but we set the B1-field to be constant everywhere.

In [ ]:
phantom = mr0.util.load_phantom(image_matrix_size)
phantom.B1[:] = 1.0

### Create the radial FLASH sequence

In [ ]:
sequence = create_seq(
    system=sys_defaults,
    test_report=False,
    timing_check=False,
    fov_xy=float(phantom.size.numpy()[0]),
    n_readout=image_matrix_size[0],
    n_spokes=int(image_matrix_size[1] * np.pi / 2),
)
sequence.write(str(fname_seq))

### Simulate the sequence
Now we pass the sequence and the phantom to the MRzero simulation and save the simulated signal as an (ISMR)MRD file.

In [ ]:
mr0_sequence = mr0.Sequence.import_file(str(fname_seq))
signal, ktraj_adc = mr0.util.simulate(mr0_sequence, phantom, accuracy=1e-2)
mr0.sig_to_mrd(fname_mrd, signal, sequence)

### Reconstruct the image
We use MRpro for the image reconstruction.

In [ ]:
kdata = KData.from_file(fname_mrd, trajectory=KTrajectoryRadial2D())
kdata.header.encoding_matrix = SpatialDimension(z=1, y=image_matrix_size[1], x=image_matrix_size[0])
kdata.header.recon_matrix = SpatialDimension(z=1, y=image_matrix_size[1], x=image_matrix_size[0])
recon = DirectReconstruction(kdata, csm=None)
idata = recon(kdata)

We can now compare the result to a simulation using the idealized signal model for spoiled gradient echo sequences.

In [ ]:
def mrzero_map_to_mrpro(mrzero_map):
    """Convert MRzero map to MRpro format (xy->yx and shift by one pixel)."""
    return np.roll(rearrange(mrzero_map.numpy().squeeze()[::-1, ::-1], 'x y -> y x'), shift=(1, 1), axis=(0, 1))


idat = idata.data.abs().numpy().squeeze()
t2star = mrzero_map_to_mrpro(1 / (1 / phantom.T2 + 1 / phantom.T2dash))
model = SpoiledGRE(flip_angle=kdata.header.flip_angle, echo_time=kdata.header.te, repetition_time=kdata.header.tr)
idat_model = model(m0=mrzero_map_to_mrpro(phantom.PD), t1=mrzero_map_to_mrpro(phantom.T1), t2star=t2star)

fig, ax = plt.subplots(1, 3, figsize=(15, 3))
for cax in ax:
    cax.set_xticks([])
    cax.set_yticks([])

im = ax[0].imshow(idat)
fig.colorbar(im, ax=ax[0], label='Input T1 (s)')

im = ax[1].imshow(idat_model)
fig.colorbar(im, ax=ax[1], label='Measured T1 (s)')

im = ax[2].imshow(idat - idat_model, cmap='bwr')
fig.colorbar(im, ax=ax[2], label='Difference T1 (s)')

relative_error = np.sum(np.abs(idat - idat_model) / np.sum(np.abs(idat_model)))
print(f'Relative error {relative_error}')
assert relative_error < 0.01